# Search Engine Demo 

Types of queries it can handle

In [1]:
# imports

from query_functions import search
from setup import startup_engine
import os

In [2]:
# paths for our data (going one directory above)
workdir = os.getcwd()
parent_path = os.path.dirname(workdir)
csv_path = parent_path + "/data/tokenized/"
import glob 
csvs = glob.glob(csv_path + "*.csv")

## Step 1: Pipeline for extracting, cleaning and tokenizing text from pdfs

### 1.1: Extracting text from pdfs

The pdfs are extracted using the `fitz` Python library. `fitz`  is a wrapper around the `PyMuPDF` library. A pipeline is created to extract text from the pdfs and save them as `.pkl` files. The pipeline is as follows:

1. Extract text from pdfs using `fitz`

2. Store the contents of each file in a dictionary with `"text"` as the key and the extracted text separated page-wise in a list as the value. (in a `.pkl` file)

3. Repeat for all pdfs and separate them in the same folder structure as the original pdfs. 

**Note:** Implementation of this step ican be found in the `cleaning.py` file. (fully documented)

### 1.2: Lemmatizing and creation of posting lists

From the created `.pkl` files we create a `DataFrame` with the following columns:

1. `id`: to uniquely identify passages ('documents' for our engine)

2. `text`: the text of the passage (split based on two newlines `\n \n` and `\n\n` and restricted to ~ 3000 characters per passage)

3. `tokenized`: the lemmatized version of the text in the paragraph (`spaCy` was used for lemmatization)

4. `posting_list`: the posting list for the passage (unique lemmas from `tokenized`)

5. `document_name`: the name of the pdf file from which the passage was extracted

6. `page_number`: the page number of the pdf from which the passage was extracted

7. `paragraph_number`: the paragraph number of the page from which the passage was extracted


**Note:** Implementation of the above steps can be found in the `tokenizing.py` file. (fully documented)

**Note 2:** The entire process outlined till here is in a pipeline, so adding new pdfs, new folders will not need any change of code except for the paths in the `cleaning.py` file.

## Step 2: Construction of inverted index and permuterm indexes

### 2.1: Inverted index

The inverted index which is used for boolean querying and ranking later on is constructed using the `posting_list` column of the `DataFrame` created in the previous step. The inverted index is a dictionary with the keys as the unique lemmas and the values as the sorted list of documents in which the lemma occurs. 

The documents are sorted based on the `id` column of the `DataFrame` and the `id` is the index of the document in the `DataFrame`.

The data structure used for this is the default Python `dict`, whose keys are `str`s and values are `LinkedList`s (implemented in in `LinkedList.py`).

**Visual example:**

```python
{
    "lemma1": [12] -> [34] -> [56] -> [78] -> [90],
    "lemma2": [1] -> [2] -> [3] -> [4] -> [5] -> [6] -> [7] -> [8] -> [9] -> [10],
}
```

### 2.2: Permuterm indexes

The permuterm indexes (normal and reverse) are constructed using the `inverted_index` created in the previous step. The permuterm indexes are dictionaries with the keys as the permuterm and the values as the sorted list of documents in which the permuterm occurs.

Like the inverted index, the documents are sorted based on the `id` column of the `DataFrame` and the `id` is the index of the document in the `DataFrame`.

**Visual example:**

1. Normal permuterm index

```python
{
    "$lemma1": [12] -> [34] -> [56] -> [78] -> [90],
    "$lemma": [12] -> [34] -> [56] -> [78] -> [90], [108],
    "$lemm" : [12] -> [34] -> [56] -> [78] -> [90], [108], [109],
    "$lem"  : [12] -> [34] -> [56] -> [78] -> [90], [108], [109], [110],
    "$le"   : [12] -> [34] -> [56] -> [78] -> [90], [108], [109], [110], [111],
    "$l"    : [12] -> [34] -> [56] -> [78] -> [90], [108], [109], [110], [111], [112],

}
```

2. Reverse permuterm index

```python
{
    "$1ammel": [12] -> [34] -> [56] -> [78] -> [90],
    "$ammel": [12] -> [34] -> [56] -> [78] -> [90], [108],
    "$mmel" : [12] -> [34] -> [56] -> [78] -> [90], [108], [109],
    "$mel"  : [12] -> [34] -> [56] -> [78] -> [90], [108], [109], [110],
    "$el"   : [12] -> [34] -> [56] -> [78] -> [90], [108], [109], [110], [111],
    "$l"    : [12] -> [34] -> [56] -> [78] -> [90], [108], [109], [110], [111], [112],
}
```

### 2.3 Biword index

The biword index is used to query `phrase` queries. It is constructed using the `tokenized` column of the `DataFrame` created in the previous step. The biword index is a dictionary with the keys as the unique biwords and the values as the sorted list of documents in which the biword occurs.

**Visual example:**

```python
{
    "lemma1 lemma2": [12] -> [34] -> [56] -> [78] -> [90],
    "lemma2 lemma3": [1] -> [2] -> [3] -> [4] -> [5] -> [6] -> [7] -> [8] -> [9] -> [10],
}
```


Implementation of the above steps can be found in the `setup.py` file. (fully documented)

**Note**: The indexes could not be saved as `.pkl` files due to the size of the indexes (hitting the maximum possible recursion limit in Python). So instead of saving these indexes, there is currently a function `start_engine` which creates and returns these indexes along with the main `DataFrame` which is used for lookups. 

In [3]:
inverted_list, perm_index, rev_perm_index, n_word_index, _, main_df = startup_engine(*csvs)

### Important statistics

While the `start_engine` function takes around `30s` to create all required indexes, this is only a 1 time run per startup, and any subsequent query will be answered in `~0.1s` (for boolean retrieval and ranking) as can be seen later on.

## Step 3: Boolean retrieval/filtering

The boolean filter can either be used:

1. Separately, if the user only wishes to filter the documents based on the query

2. As a precursor to the ranking algorithm, if the user wishes to rank the documents based on the query, so that we wont have to rank documents which do not match the query. 

### 3.1: Free queries

I've implemented querying of:

1. `OR` queries: by default all words passed in the query string are treated as `OR` queries. For example, the query `information retrieval` will return all documents which contain either `information` or `retrieval` or both.

2. `AND` queries: If any words in the query string are enclosed in double quotes, they are treated as `AND` queries. For example, the query `"information retrieval"` will return all documents which contain both `information` and `retrieval`. All other words not enclosed in double quotes (the `OR` parts of the query) won't be considered in the filtering. The ranking algorithm however will consider all words in the query.

In [4]:
def user_search(query, ranked=True, is_phrase=False, summarize=False, num_docs=None, spell_check=False, autocomplete=False, n_auto_results=5):
    """Simpler function to call the engine's search function without passing in the indexes
    """
    return search(query, inverted_list, perm_index, rev_perm_index, n_word_index, main_df, ranked=ranked, is_phrase=is_phrase, show_summary=summarize, retrieve_n = num_docs, spell_check=spell_check, autocomplete=autocomplete, n_auto_results=n_auto_results)

In [5]:
# Defaults to a ranked `OR` query
user_search("protection officer", num_docs=1)    

Documents Retrieved: 1
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
Rank: 1
Document Name: complete-property-owner-policy-wording-policies-incepting-or-renewing-from-010418-acom686-11
Page Number: 63
Paragraph Number: 1
Score: 21.654312876275398
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
Paragraph Text: 
61Complete Property Owner Policy Wording9Know your rightsAny individual whose personal information we hold has theright to:• object to us processing it. We will either agree to stopprocessing or explain why we are unable to (the right toobject)• ask for a copy of their personal information we hold, subjectto certain exemptions (a data subject access request)• ask us to update or correct their personal information t

In [6]:
# Unranked `OR` query
user_search("protection detailed data", ranked=False, num_docs=1)

Documents Retrieved: 1
Unranked Search Results: Boolean Retrieval
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
Rank: 1
Document Name: 5105072011_booklet
Page Number: 6
Paragraph Number: 1
Score: None
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
Paragraph Text: 
SAMPLE DOCUMENT2 damages are directly caused by actual physical injury to the person claiming damages. Business – means any full or part time activity from which any insured may derive an economic benefit, regardless of profit or loss. Business includes, but is not limited to, commercial enterprise, trade, hobby, profession, occupation or employment, or the renting, leasing or holding for rental or lease of any part of any premises by any insured. If an insure

In [7]:
# Ranked `AND` query
user_search("\"protection\" officer", num_docs=1)

Documents Retrieved: 1
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
Rank: 1
Document Name: complete-property-owner-policy-wording-policies-incepting-or-renewing-from-010418-acom686-11
Page Number: 63
Paragraph Number: 1
Score: 21.654312876275398
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
Paragraph Text: 
61Complete Property Owner Policy Wording9Know your rightsAny individual whose personal information we hold has theright to:• object to us processing it. We will either agree to stopprocessing or explain why we are unable to (the right toobject)• ask for a copy of their personal information we hold, subjectto certain exemptions (a data subject access request)• ask us to update or correct their personal information t

### 3.2: Phrase queries

The phrase queries are implemented using the biword index. The query string is split into biwords and the documents which contain all the biwords are returned. To set your query as a phrase query/ give importance to the order of the words in the query, set the `is_phrase` parameter to `True` in the `search` function.


In [8]:
# Ranked `phrase` query
user_search("Protection Officer", is_phrase=True, num_docs=1)

Documents Retrieved: 1
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
Rank: 1
Document Name: complete-property-owner-policy-wording-policies-incepting-or-renewing-from-010418-acom686-11
Page Number: 63
Paragraph Number: 1
Score: 21.654312876275398
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
Paragraph Text: 
61Complete Property Owner Policy Wording9Know your rightsAny individual whose personal information we hold has theright to:• object to us processing it. We will either agree to stopprocessing or explain why we are unable to (the right toobject)• ask for a copy of their personal information we hold, subjectto certain exemptions (a data subject access request)• ask us to update or correct their personal information t

### 3.3: Wildcard matching

Wildcards are automatically detected in the query string and the documents which contain the wildcard are returned. The wildcard can be placed anywhere in the query string. For example, the query `infor*` will return all documents which contain words starting with `infor`. The query `*tion` will return all documents which contain words ending with `tion`. The query `inf*tion` will return all documents which contain words starting with `inf` and ending with `tion`.

As of now, only one wildcard is supported in each query word of the string. As in `p*y*n d*iv*` will be translated to `p*n d*` in the backend.

In [9]:
# Type 1 wildcard query
user_search("prot* *tion", num_docs=1)

Documents Retrieved: 1
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
Rank: 1
Document Name: Property-Owner-Policy-Wording
Page Number: 66
Paragraph Number: 1
Score: 276.5811681853387
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
Paragraph Text: 
64 | Complete Property Owners InsuranceNuclear InstallationAny installation of such class or description as may beprescribed by regulations made by the relevant Secretaryof State from time to time by statutory instrument, beingan installation designed for or adapted for1 the production or use of atomic energy or2 the carrying out of any process which is preparatory orancillary to the production or use of atomic energy andwhich involves or is capable of causing the emission ofio

In [10]:
# Type 2 wildcard query
user_search("pro*ion", num_docs=1)

Documents Retrieved: 1
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
Rank: 1
Document Name: rsa_property_owners_policy_wording
Page Number: 40
Paragraph Number: 1
Score: 24.821053129151498
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
Paragraph Text: 
40 | Properties Policysection 5 Data protectionWhat is coveredWhat is not covered1 The defence of any Legal proceedings brought against You for compensation under Section 13 of the Data Protection Act 1998 provided that You are already registered with the Data Protection Commissioner2 An appeal by You againstA) the refusal of Your application for registration by the Data Protection CommissionerB) the refusal of an application for alteration of registered particulars by th

In [11]:
# Wildcard query with `AND`
user_search("\"pro*ion\" officer", num_docs=1)

Documents Retrieved: 1
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
Rank: 1
Document Name: rsa_property_owners_policy_wording
Page Number: 40
Paragraph Number: 1
Score: 24.821053129151498
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
Paragraph Text: 
40 | Properties Policysection 5 Data protectionWhat is coveredWhat is not covered1 The defence of any Legal proceedings brought against You for compensation under Section 13 of the Data Protection Act 1998 provided that You are already registered with the Data Protection Commissioner2 An appeal by You againstA) the refusal of Your application for registration by the Data Protection CommissionerB) the refusal of an application for alteration of registered particulars by th

**Note**: Phrase queries get converted from an `AND` to an `OR` query if wildcards are used

In [12]:
# Wildcard with phrase query
user_search("\"pro*ion\" \"officer\"", is_phrase=True, num_docs=1)

Documents Retrieved: 1
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
Rank: 1
Document Name: BRIT-PO-Policy-Wording-May-2016-1
Page Number: 53
Paragraph Number: 1
Score: 24.821053129151498
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
Paragraph Text: 
 Page | 53 Further Information Data Protection Act 1998 In order to assess the terms of the insurance contract or administer claims that arise, We may need to collect data that the Data Protection Act defines as personal or sensitive. All data collected, including personal and sensitive data, will be kept secure at all times in accordance with the provisions of Data Protection Act 1998. We will also monitor and record Our communication with You for compliance and training 

### 3.4: Spelling correction

The engine first assumes that the spelling is correct. But if no documents are returned by the boolean filter, the engine finds the closest words to all non wildcard query words and returns the results of an `OR` query of all the closest words. Spell checking is turned off by default, but can be turned on by setting the `spell_check` parameter to `True` in the `search` function.

Spell checking was implemented using `levenshtein` distance. In addition to the general `insert, delete and replace` operations, I've implemented a `swap` operation which is used to correct words like `inforamtion` to `information`, since this kind of spelling error is very common.

Implementation of the above steps can be found in the `edit_distance_functions.py` file. (fully documented)

In [13]:
# Without spell check
user_search("\"prtoection\" officer", num_docs=1)

No documents found


In [14]:
# With spell check
user_search("\"prtoection\" officer", spell_check=True, num_docs=1)

No documents found with direct match with "prtoection" officer. Performing spell check...
Corrected Query: protection officer
Documents Retrieved: 1
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
Rank: 1
Document Name: complete-property-owner-policy-wording-policies-incepting-or-renewing-from-010418-acom686-11
Page Number: 63
Paragraph Number: 1
Score: 21.654312876275398
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
Paragraph Text: 
61Complete Property Owner Policy Wording9Know your rightsAny individual whose personal information we hold has theright to:• object to us processing it. We will either agree to stopprocessing or explain why we are unable to (the right toobject)• ask for a copy of their personal information w

### 3.5: Autocomplete suggestions

To mimic the autocomplete feature of Google search, I've implemented a feature which returns the top 5 most frequent words in the documents which contain the query string. This is turned off by default, but can be turned on by setting the `autocomplete` parameter to `True` in the `search` function.

It works by finding the words in the corpus with the least distance from the last word in the query string as well as their frequencies.

The lemmatized words are 'un'lemmatized for human readabiltiy. This is achieved using the `lemminflect` package which goes hand in hand with the package I used for lemmatization (`spaCy`).

In [15]:
user_search("protection off", autocomplete=True, n_auto_results=10)

Possible Options:
------------------------------------------------------------------------------------------
1. protection offers
2. protection offer
3. protection offered
4. protection offering
5. protection offices
6. protection office
7. protection offsets
8. protection offset
9. protection off set
10. protection off setted
------------------------------------------------------------------------------------------


## Step 4: Ranking of documents

### 4.1: Opting out of ranking

As mentioned earlier, the ranking algorithm can be turned off if all that is required is a boolean filter. To turn off the ranking algorithm, set the `ranked` parameter to `False` in the `search` function.

### 4.2: TF-IDF scores

The ranking is done based on the TF-IDF scores of the query words in the documents. The TF-IDF scores are calculated using the `inverted_index` created in the previous step. The TF-IDF scores are calculated for each word in the query string and the final score of a document is the sum of the TF-IDF scores of all the words in the query string.

it is calculated document at a time, and in the case of wildcard queries, the TF-IDF scores are calculated for each word in the document which matches the wildcard.

Implementation of the above steps can be found in the `scoring_functions.py` file. (fully documented)

## Extra: Summarizing results

Using a pre-trained `Bart` model, I've implemented a feature which summarizes the results of the query. The summary is returned as a string along with the results of the query. To turn on the summarization feature, set the `summarize` parameter to `True` in the `search` function.

Summarization of results however takes a very long time to run and is dependent on the length of the results. So it is turned off by default.

In [16]:
user_search("\"prtoection\"", summarize=True, num_docs=1, spell_check=True)

No documents found with direct match with "prtoection". Performing spell check...
Corrected Query: protection
Documents Retrieved: 1
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
Rank: 1
Document Name: Property-Owner-Policy-Wording
Page Number: 36
Paragraph Number: 1
Score: 13.993896691013797
------------------------------------------------------------------------------------------
Summary:  The total amount payable including all costs andexpenses in respect of all claims occurring during anyone Period of Insurance, is limited to £100,000 . The Insurer will indemnify the Insured and at theInsured’s request any partner, director or Employee ofthe Insured against the sums which the Insurer or any director, partner or Employee become(s)legally liable to pay as compensation, under the Data Protection Act 1984 .
---------------------------------------------